In [9]:
import os
import pandas as pd
import numpy as np


base_dir = r"C:\Users\mh272\Box\NED3_Share\Ishraq Hossain\CWRU\test17_17th Oct_Friday\Flow Loop Dataset"

# You specify the output name:
final_name = input("Enter the final Excel filename (e.g., Final_Run_17.xlsx): ").strip()

# Use name without extension as the stem
stem = os.path.splitext(final_name)[0]   # e.g., "Final_Run_17"

# Build paths
path_U = os.path.join(base_dir, f"Danial_test_{stem}.xlsx")
path_Z = os.path.join(base_dir, f"Danial_test_{stem}P.xlsx")

# (Optional) quick existence check
if not os.path.exists(path_U) or not os.path.exists(path_Z):
    raise FileNotFoundError("One or both Excel files not found in base_dir.")

# Load exactly with pandas
# pd.read_excel only, but explicitly use openpyxl with safe kwargs
U = pd.read_excel(path_U, engine="openpyxl",
                  engine_kwargs={"read_only": True, "data_only": True})
Z = pd.read_excel(path_Z, engine="openpyxl",
                  engine_kwargs={"read_only": True, "data_only": True})

print(f"U shape: {U.shape}")
print(f"Z shape: {Z.shape}")

U shape: (1330, 17)
Z shape: (1330, 4)


In [10]:
def FC72_P_Tsat(p):
    return -9.92201274365393e-32*p**6 + 1.91211199556047e-25*p**5 - 1.44614437956564e-19*p**4 + \
           5.46964864114115e-14*p**3 - 1.1085901689793e-8*p**2 + 1.33812744556323e-3*p - 7.54613965236922

def FC72_T_hf(T):
    return 1.63974456582339e-9*T**6 - 4.08810469920695e-7*T**5 + 4.48924627107772e-5*T**4 - \
           1.26619121874683e-3*T**3 + 0.866823909804224*T**2 + 1003.83556473627*T - 59997.2511652559

def FC72_T_hg(T):
    return -9.66157980009097e-10*T**6 + 2.37945375403115e-7*T**5 - 2.81138727586949e-5*T**4 + \
           2.17312888707966e-4*T**3 + 0.966423372738063*T**2 + 718.110564477741*T + 39699.0573808848

def FC72_T_cpf(T):
    return 1.44871633461253e-10*T**6 - 3.58026652769183e-8*T**5 + 3.63919204104945e-6*T**4 - \
           1.61701151455418e-4*T**3 + 6.10214733751491e-3*T**2 + 1.59506874892395*T + 1004.18123037088
# Extract columns from the data
P = Z.iloc[:, 3]  # (Watt)
V = Z.iloc[:, 1]  #  (Voltage)
M = U.iloc[:, 1]  #  mass flow rate g/s
F = U.iloc[:, 1] / 1000  # 2nd column, mass flow rate kg/s
W=2.5*10**-3 # width of channel meter
H=5*10**-3  # height of channel meter
L = 0.1146  # Length (meters)
A=W*H # Cross section of channel in meter square
A_h=L*W # heater area
G = M/A  # Mass velocity of flow kg/m2.s

rho = 1680  # Density kg/m3
mu = 0.00064  # Dynamic viscosity (Pa.s)
Vs = F / (rho * A)  # velocity
dh = 1 / 300  # 
alpha = 2.5 / 5.0  # aspect ratio

# Time 
time = U.iloc[:,0]
# 2) make sure it's datetime (safe if it already is)
time = pd.to_datetime(time)

# 3) set t0 as the very first timestamp
t0 = time.iloc[0]

# 4) relative time columns
U['t_rel_s']  = (time - t0).dt.total_seconds().round(3)        # e.g., 0.000, 0.100, ...
U['t_rel_ms'] = ((time - t0).dt.total_seconds()*1000).round(1) # in milliseconds
U['t_rel_str'] = U['t_rel_s'].map(lambda x: f"{x:.3f}")  

real_time = U['t_rel_s']

# Temperatures from multiple columns in U (unit: assumed °C or K)
T1 = U.iloc[:, 4]
T2 = U.iloc[:, 5]
T3 = U.iloc[:, 6]
T4 = U.iloc[:, 7]
T5 = U.iloc[:, 8]
T6 = U.iloc[:, 9]
T7 = U.iloc[:, 10]

# Other values
Tin = U.iloc[:, 11]  # Inlet temperature (unit: assumed °C or K)
Tout = U.iloc[:, 13]  # Outlet temperature (unit: assumed °C or K)
Pin = U.iloc[:, 14] * 6.8947 * 1000  # Convert psi to Pa (unit: Pascals, Pa)
Pout = U.iloc[:, 15] * 6.8947 * 1000  # Convert psi to Pa (unit: Pascals, Pa)
dP = Pin - Pout  # Pressure drop (unit: Pascals, Pa)

# Average temperature (unit: °C or K)
Ta = (T1 + T2 + T3 + T4 + T5 + T6 + T7) / 7


# Coordinate positions (unit: meters)
x1 = 0.0054  # (unit: meters)
x2 = 0.0227  # (unit: meters)
x3 = 0.04    # (unit: meters)
x4 = 0.0573  # (unit: meters)
x5 = 0.0746  # (unit: meters)
x6 = 0.0919  # (unit: meters)
x7 = 0.1092  # (unit: meters)

Qm = P / A_h # total heat flux
Tsat_in = FC72_P_Tsat(Pin)
Tsat_out = FC72_P_Tsat(Pout)
hfin = FC72_T_hf(Tsat_in)
hgin = FC72_T_hg(Tsat_in)
Cpfin = FC72_T_cpf(Tsat_in)
hfgin = hgin - hfin  
Tsub_in = Tsat_in - Tin
Tsub_out = Tsat_out - Tout
xin = (-Cpfin * (Tsat_in-Tin)) / hfgin
xout = xin + ((P * W) / A_h) / (F * hfgin) * L
xf1 = xin + (Qm * W * x1) / (F * hfgin)
xf2 = xin + (Qm * W * x2) / (F * hfgin)
xf3 = xin + (Qm * W * x3) / (F * hfgin)
xf4 = xin + (Qm * W * x4) / (F * hfgin)
xf5 = xin + (Qm * W * x5) / (F * hfgin)
xf6 = xin + (Qm * W * x6) / (F * hfgin)
xf7 = xin + (Qm * W * x7) / (F * hfgin)
# Coordinate positions
x_positions = [x1, x2, x3, x4, x5, x6, x7]
T_values = [T1, T2, T3, T4, T5, T6, T7]
xf_values = [xf1, xf2, xf3, xf4, xf5, xf6, xf7]  # Assuming these are Series or arrays

# Initialize lists to store results
pf_values = []
Tf_values = []
h_values = []

# Iterate over the positions and temperatures
for i, (x, T, xf) in enumerate(zip(x_positions, T_values, xf_values)):
    pf = Pin + ((Pout - Pin) / L) * x

    # Use vectorized condition for `xf < 0`
    if (xf < 0).all():  # Check if all elements in the series satisfy xf < 0
        if i == 0:
            Tf = Tin + (P / L) * x / (F * Cpfin)
        else:
            Tf = Tin + (P / L) * x / (F * Cpfin)
    else:
        Tf = FC72_P_Tsat(pf)

    h = Qm / (T - Tf)
    
    # Store results
    pf_values.append(pf)
    Tf_values.append(Tf)
    h_values.append(h)

# Now, `pf_values`, `Tf_values`, and `h_values` contain the results for all positions
# Assuming all variables (T1, T2, ..., etc.) are numpy arrays or pandas series

# Extract h1 to h7 from h_values
h_values = np.array(h_values)  # Convert to numpy array
h1 = h_values[0]
h2 = h_values[1]
h3 = h_values[2]
h4 = h_values[3]
h5 = h_values[4]
h6 = h_values[5]
h7 = h_values[6]

# Temperature differences
dT = T7 - T1
dT_values = [T1 - Tsat_in, T2 - Tsat_in, T3 - Tsat_in, T4 - Tsat_in, T5 - Tsat_in, T6 - Tsat_in, T7 - Tsat_in]
dT1, dT2, dT3, dT4, dT5, dT6, dT7 = dT_values
dTw = Ta - Tsat_in

# Average heat transfer coefficient
havg = (h1 + h2 + h3 + h4 + h5 + h6 + h7) / 7

# Correct Q calculation (Q is heat flux)
Q = P / (L * W)  # P is power in Watts, L and W are dimensions in meters

# delta T
delT = Tout - Tin

# Calculating Qsp, dq, qq
Qsp = F * Cpfin * np.abs(delT) * 0.53  # Adjust the factor if necessary
dq = np.abs(P - Qsp)
qq = (dq / P) * 100

# Reynolds number and friction factor calculations
Re = (rho * Vs * dh) / mu  # Vs in m/s, dh in m, mu in Pa.s

f = 0.3164 / (Re ** 0.25)  # Blasius formula

Retu = Re * (2 * alpha) ** 0.16
ftu = 0.3164 / (Retu ** 0.25)  # Troniewski-Ulbrich

# Other friction factor calculations
alpha_term = alpha
cv = 96 * (1 - (1.3553 * alpha_term) + (1.9467 * alpha_term ** 2) - (1.7012 * alpha_term ** 3) + \
           (0.9546 * alpha_term ** 4) - (0.2537 * alpha_term ** 5))
fs = f * ((((0.0154 * cv) / 64) - 0.012) ** (1 / 3) + 0.85)  # Sadatomi formula
fk = f * (1.0875 - 0.1125 * alpha_term)  # Kakac-Yener

# Bhatti-Shah friction factor
fb = np.where(Re < 4000,
              0.0216 + (9.2e-8) * (Re ** 1.5),
              5.12e-3 + 0.4572 * Re ** -0.311)

fbs = fb * (1.0875 - 0.1125 * alpha_term)

# Kandlikar friction factor
Rek = Re * (0.67 + 0.458 * alpha_term * (2 - alpha_term))
fka = 0.12792 * Rek ** -0.28

# Experimental friction factor
fexp = (dP * 2 * rho * dh) / (G ** 2 * L)

# Writing data to Excel
df1 = pd.DataFrame({
    'Time [h,m,s]': time,
    'Time [s]': real_time,
    'Mass Flow Rate (M) [g/s]': M,
    'Fluid Velocity (Vs) [m/s]': Vs,
    'Inlet Temperature (Tin) [°C]': Tin,
    'Outlet Temperature (Tout) [°C]': Tout,
    'Inlet Pressure (Pin) [Pa]': Pin,
    'Outlet Pressure (Pout) [Pa]': Pout,
    'Voltage (V) [V]': V,
    'Power (P) [W]': P,
    'Temperature at x1 (T1) [°C]': T1,
    'Temperature at x2 (T2) [°C]': T2,
    'Temperature at x3 (T3) [°C]': T3,
    'Temperature at x4 (T4) [°C]': T4,
    'Temperature at x5 (T5) [°C]': T5,
    'Temperature at x6 (T6) [°C]': T6,
    'Temperature at x7 (T7) [°C]': T7,
    'Average Temperature (Ta) [°C]': Ta,
    'Temperature Difference (dT) [°C]': dT,
    'Saturation Temperature Inlet (Tsat_in) [°C]': Tsat_in,
    'Subcooling Inlet (Tsub_in) [°C]': Tsub_in,
    'Saturation Temperature Outlet (Tsat_out) [°C]': Tsat_out,
    'Subcooling Outlet (Tsub_out) [°C]': Tsub_out,
    'Inlet Vapor Quality (xin)': xin,
    'Vapor Quality at x1 (xf1)': xf1,
    'Vapor Quality at x2 (xf2)': xf2,
    'Vapor Quality at x3 (xf3)': xf3,
    'Vapor Quality at x4 (xf4)': xf4,
    'Vapor Quality at x5 (xf5)': xf5,
    'Vapor Quality at x6 (xf6)': xf6,
    'Vapor Quality at x7 (xf7)': xf7,
    'Outlet Vapor Quality (xout)': xout,
    'Heat Transfer Coefficient at x1 (h1) [W/m²·K]': h1,
    'Heat Transfer Coefficient at x2 (h2) [W/m²·K]': h2,
    'Heat Transfer Coefficient at x3 (h3) [W/m²·K]': h3,
    'Heat Transfer Coefficient at x4 (h4) [W/m²·K]': h4,
    'Heat Transfer Coefficient at x5 (h5) [W/m²·K]': h5,
    'Heat Transfer Coefficient at x6 (h6) [W/m²·K]': h6,
    'Heat Transfer Coefficient at x7 (h7) [W/m²·K]': h7,
    'Total Heat Flux (Qm) [W/m²]': Qm,
    'Temperature Difference (dTw) [°C]': dTw,
    'Heat Flux (Q) [W/m²]': Q,
    'Average Heat Transfer Coefficient (havg) [W/m²·K]': havg
})

df2 = pd.DataFrame({
    'Time [h,m,s]': time,
    'Time [s]': real_time,
    'Mass Flow Rate (M) [g/s]': M,
    'Power (P) [W]': P,
    'Calculated Heat Rate (Qsp) [W]': Qsp,
    'Heat Rate Difference (dq) [W]': dq,
    'Percentage Difference (qq) [%]': qq,
    'Experimental Friction Factor (fexp)': fexp,
    'Friction Factor (f)': f,
    'Friction Factor (Troniewski-Ulbrich, ftu)': ftu,
    'Friction Factor (Sadatomi, fs)': fs,
    'Friction Factor (Kakac-Yener, fk)': fk,
    'Friction Factor (Bhatti-Shah, fbs)': fbs,
    'Friction Factor (Kandlikar, fka)': fka
})



# You specify the output name:
#final_name = input("Enter the final Excel filename (e.g., Final_Run_17.xlsx): ").strip()

# ensure .xlsx and get stem
if not final_name.lower().endswith(".xlsx"):
    final_name += ".xlsx"
final_stem = os.path.splitext(final_name)[0]  # e.g., "Final_Run_17"

# build output filenames
out1 = os.path.join(base_dir, f"{final_stem}_IN_with_units.xlsx")
out2 = os.path.join(base_dir, f"{final_stem}_u_with_units.xlsx")

os.makedirs(base_dir, exist_ok=True)

# Ensure datetime column is true datetime and rounded to milliseconds
df1['Time [h,m,s]'] = pd.to_datetime(df1['Time [h,m,s]']).dt.round('ms')
df2['Time [h,m,s]'] = pd.to_datetime(df2['Time [h,m,s]']).dt.round('ms')

# ---- save df1 ----
with pd.ExcelWriter(out1, engine="xlsxwriter",
                    datetime_format="yyyy-mm-dd hh:mm:ss.000") as w:
    df1.to_excel(w, index=False, sheet_name="Sheet1")
    ws = w.sheets["Sheet1"]
    fmt = w.book.add_format({'num_format': 'yyyy-mm-dd hh:mm:ss.000'})
    c = df1.columns.get_loc('Time [h,m,s]')
    ws.set_column(c, c, 23, fmt)  # width 23 + number format

# ---- save df2 ----
with pd.ExcelWriter(out2, engine="xlsxwriter",
                    datetime_format="yyyy-mm-dd hh:mm:ss.000") as w:
    df2.to_excel(w, index=False, sheet_name="Sheet1")
    ws = w.sheets["Sheet1"]
    fmt = w.book.add_format({'num_format': 'yyyy-mm-dd hh:mm:ss.000'})
    c = df2.columns.get_loc('Time [h,m,s]')
    ws.set_column(c, c, 23, fmt)


# save
#df1.to_excel(out1, index=False)
#df2.to_excel(out2, index=False)

print("✅ Saved:")
print(" -", out1)
print(" -", out2)



✅ Saved:
 - C:\Users\mh272\Box\NED3_Share\Ishraq Hossain\CWRU\test17_17th Oct_Friday\Flow Loop Dataset\25gs_20C_IN_with_units.xlsx
 - C:\Users\mh272\Box\NED3_Share\Ishraq Hossain\CWRU\test17_17th Oct_Friday\Flow Loop Dataset\25gs_20C_u_with_units.xlsx


In [11]:
time

0      2025-10-17 13:51:31.605
1      2025-10-17 13:51:31.705
2      2025-10-17 13:51:31.805
3      2025-10-17 13:51:31.905
4      2025-10-17 13:51:32.005
                 ...          
1325   2025-10-17 14:01:38.105
1326   2025-10-17 14:01:38.205
1327   2025-10-17 14:01:38.305
1328   2025-10-17 14:01:38.405
1329   2025-10-17 14:01:38.505
Name: Time, Length: 1330, dtype: datetime64[ns]

In [12]:
# Convert total heat flux to W/cm²
Qm_cm2 = Qm * 1e-4  # Convert from W/m² to W/cm²

# Convert pressures to kPa
Pin_kPa = Pin / 1000  # Convert from Pa to kPa
Pout_kPa = Pout / 1000  # Convert from Pa to kPa

# Prepare the DataFrame with the required data
df_final = pd.DataFrame({
    'Time [h,m,s]': time,
    'Time [s]': real_time,
    'Mass Flow Rate [kg/s]': F,
    'Fluid Velocity [m/s]': Vs,
    'Power [W]': P,
    'Total Heat Flux [W/cm²]': Qm_cm2,
    'Inlet Subcooling [°C]': Tsub_in,
    'Inlet Pressure [kPa]': Pin_kPa,
    'Outlet Pressure [kPa]': Pout_kPa,
    'Inlet Temperature [°C]': Tin,
    'Outlet Temperature [°C]': Tout,
    'Saturation Temperature Inlet [°C]': Tsat_in,
    'Saturation Temperature Outlet [°C]': Tsat_out,
    'Location Inlet [m]': 0.0,
    'Location x1 [m]': x1,
    'Location x2 [m]': x2,
    'Location x3 [m]': x3,
    'Location x4 [m]': x4,
    'Location x5 [m]': x5,
    'Location x6 [m]': x6,
    'Location x7 [m]': x7,
    'Location Outlet [m]': L,
    'Temperature at Inlet [°C]': Tin,
    'Temperature at x1 [°C]': T1,
    'Temperature at x2 [°C]': T2,
    'Temperature at x3 [°C]': T3,
    'Temperature at x4 [°C]': T4,
    'Temperature at x5 [°C]': T5,
    'Temperature at x6 [°C]': T6,
    'Temperature at x7 [°C]': T7,
    'Temperature at Outlet [°C]': Tout,
    'Quality at Inlet': xin,
    'Quality at x1': xf1,
    'Quality at x2': xf2,
    'Quality at x3': xf3,
    'Quality at x4': xf4,
    'Quality at x5': xf5,
    'Quality at x6': xf6,
    'Quality at x7': xf7,
    'Quality at Outlet': xout,
    'Heat Transfer Coefficient at x1 [W/m²·K]': h1,
    'Heat Transfer Coefficient at x2 [W/m²·K]': h2,
    'Heat Transfer Coefficient at x3 [W/m²·K]': h3,
    'Heat Transfer Coefficient at x4 [W/m²·K]': h4,
    'Heat Transfer Coefficient at x5 [W/m²·K]': h5,
    'Heat Transfer Coefficient at x6 [W/m²·K]': h6,
    'Heat Transfer Coefficient at x7 [W/m²·K]': h7,
})


'''
# Get the base name of the original file without extension
original_file_name = '5G6C.xlsx'  # Replace with your actual file variable if it's dynamic
base_name = os.path.splitext(original_file_name)[0]

# Create the final file name
final_file_name = f'Finall{base_name}.xlsx'

# Save the DataFrame to an Excel file
df_final.to_excel(final_file_name, index=False)

print(f"Data has been successfully saved to '{final_file_name}'.")



# ---- Save df_final to the same base directory ----
# base_dir must already be defined earlier in your script

# You specify the output name:
#final_name = input("Enter the final Excel filename (e.g., Final_Run_17.xlsx): ").strip()

# Ensure .xlsx extension
if not final_name.lower().endswith(".xlsx"):
    final_name += ".xlsx"

# Build full path in base_dir
final_path = os.path.join(base_dir, final_name)

# Create folder if it somehow doesn't exist (safe no-op if it does)
os.makedirs(base_dir, exist_ok=True)

# Save
df_final.to_excel(final_path, index=False)
print(f"✅ Data has been successfully saved to:\n{final_path}")

'''

# Ensure the time column is true datetime rounded to ms
df_final['Time [h,m,s]'] = pd.to_datetime(df_final['Time [h,m,s]']).dt.round('ms')

# Build full path
if not final_name.lower().endswith(".xlsx"):
    final_name += ".xlsx"
final_path = os.path.join(base_dir, final_name)
os.makedirs(base_dir, exist_ok=True)

# Save with format applied ONLY to the first column
with pd.ExcelWriter(final_path, engine="xlsxwriter",
                    datetime_format="yyyy-mm-dd hh:mm:ss.000") as w:
    df_final.to_excel(w, index=False, sheet_name="Sheet1")
    ws = w.sheets["Sheet1"]
    wb = w.book

    # Number format for datetime with milliseconds
    dt_fmt = wb.add_format({'num_format': 'yyyy-mm-dd hh:mm:ss.000'})

    # Column 0 is the first column in the sheet (Time [h,m,s])
    ws.set_column(0, 0, 23, dt_fmt)

print(f"✅ Data has been successfully saved to:\n{final_path}")

✅ Data has been successfully saved to:
C:\Users\mh272\Box\NED3_Share\Ishraq Hossain\CWRU\test17_17th Oct_Friday\Flow Loop Dataset\25gs_20C.xlsx
